# Import the Required Modules

In [3]:
import cv2
import PIL
import numpy as np
import tkinter as tk
import os
import imutils
from imutils.contours import sort_contours
from tkinter import filedialog
from keras.models import load_model

# Application

In [ ]:
#Load the GUI
root = tk.Tk()
root.withdraw()

def GetPaths():
    seed_folder = r"C:\Users\Dell\OneDrive\Desktop"

    #Using tkinter we ask for the file path & where is should stored
    img_path = filedialog.askopenfilename(initialdir = seed_folder, title = "Select the image to be textualized", filetypes = [("image",".jpg .jpeg .png")])
    save_folder_path = filedialog.askdirectory(initialdir = seed_folder, title = "Select a folder to save processed image")

    return img_path, save_folder_path

#Load the Image and Save Folder from User
img_path, save_folder_path = GetPaths()

#Load the Model
model = load_model('handwriting.model')

# Load Image
image = cv2.imread(img_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Detect contours and sort from left to right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]

# Contour Processing
chars = []
for c in cnts:
	# Identify the bounding box of the contours
	(x, y, w, h) = cv2.boundingRect(c)
	if (w >= 20 and w <= 150) and (h >= 40 and h <= 120):
		roi = gray[y:y + h, x:x + w]
		thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
		(tH, tW) = thresh.shape

		# Resize
		if tW > tH:
			thresh = imutils.resize(thresh, width=32)
		else:
			thresh = imutils.resize(thresh, height=32)

		# Pad the Image
		(tH, tW) = thresh.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)
		padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,value=(0, 0, 0))
		padded = cv2.resize(padded, (32, 32))

		# Process Padded Image
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)
		chars.append((padded, (x, y, w, h)))
boxes = [b[1] for b in chars]

# Perform OCR
chars = np.array([c[0] for c in chars], dtype="float32")
preds = model.predict(chars)

# Decode Labels
labelNames = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]
for (pred, (x, y, w, h)) in zip(preds, boxes):
    i = np.argmax(pred)
    prob = pred[i]
    label = labelNames[i]

    # Draw the prediction on Screen
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, label, (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

# Display the Image
cv2.imshow("Textualizer_Processed.jpg",image)
cv2.waitKey(0)

#Save the Image           
cv2.imwrite(save_folder_path + "_Processed.jpg", image)